In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from zero_helpers.imports import * 
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.helpers import base_config
from zero_liftsim.experience import AgentRideLoopExperience
from zero_liftsim.helpers import load_asset_template
from zero_liftsim import helpers as hp
from zero_liftsim.lift import Lift

import inspect

# Run Simulation

In [3]:
cfg = base_config()
cfg["SimulationManager"]["__init__"].update({"n_agents": 3, "lift_capacity": 2})
manager = SimulationManager(cfg)
result = manager.run(runtime_minutes=60)
print(result)

{'total_rides': 13, 'average_wait': 2.076923076923077, 'agents': [Agent(1) nutty-science d895e842-fd97-4212-b3cd-c764331de23c, Agent(2) tasty-pitch 94dbfaa4-97a6-4a2d-b928-eccae4938419, Agent(3) squealing-purchase 43631671-c852-4f3a-b291-ebfc1587e1dd]}


In [4]:
lift = manager.lift
lift.total_chairs()

50

## Retrieve Data from Simulation

In [5]:
# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
exp = exp_log_data['exp_rideloop']
log = exp_log_data['agent_log']

# Dev

In [6]:
# sample agent and subset exp/log to e/l for agent subset of those
m = log['event'] == 'ride_complete'
event_ride_comp = log[m].sample().iloc[0].to_dict()
agent = manager.lookup_agent(event_ride_comp['agent_uuid'])
l = log[log['agent_uuid'] == agent.agent_uuid]
idx = (exp['time'] - event_ride_comp['time']).dt.total_seconds().abs().idxmin()
e = exp.loc[idx].to_dict()

In [7]:
e

{'time': Timestamp('2025-03-12 09:31:00'),
 'exp_id': '8e82b9f8-95e1-4d2c-8526-dda1139fa307',
 'return_event_uuid': 'fd5cfa54-8a01-4eb5-8ec0-cfd3d4cd5295',
 'agent_uuid': '94dbfaa4-97a6-4a2d-b928-eccae4938419',
 'agent_uuid_codename': 'tasty-pitch',
 'time_spent_ride_lift': 6.071562547769156,
 'time_spent_traverse_down_mountain': 4.961262873092367,
 'time_spent_in_queue': 5,
 'time_spent_total': 16.032825420861524}

In [8]:
x = agent.traceback_experience(e['exp_id'])
print(x)

# Experience Log 

exp_id: 8e82b9f8-95e1-4d2c-8526-dda1139fa307
agent_uuid: 94dbfaa4-97a6-4a2d-b928-eccae4938419
return_event_uuid: fd5cfa54-8a01-4eb5-8ec0-cfd3d4cd5295

*** 


time_spent_in_queue: 5
time_spent_ride_lift: 6.071562547769156
time_spent_traverse_down_mountain: 4.961262873092367

***

# Full Dict

{
    "exp_id": "8e82b9f8-95e1-4d2c-8526-dda1139fa307",
    "return_event_uuid": "fd5cfa54-8a01-4eb5-8ec0-cfd3d4cd5295",
    "agent_uuid": "94dbfaa4-97a6-4a2d-b928-eccae4938419",
    "agent_uuid_codename": "tasty-pitch",
    "time_spent_ride_lift": 6.071562547769156,
    "time_spent_traverse_down_mountain": 4.961262873092367,
    "time_spent_in_queue": 5,
    "time_spent_total": 16.032825420861524
}

# Agent Log

{
    "77611d0d-ca38-490e-8d2e-217db9616b6b": {
        "return_event_uuid": "4399fa18-a7ef-466d-825a-fc0120fdfc64",
        "time": "2025-03-12T09:13:00",
        "time_offset": 13,
        "event": "ride_complete",
        "agent_id": 2,
        "agent_uuid": "94dbfaa4